In [16]:
# import h5 reader
import h5py
import numpy as np
work_path = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/bar_shearwave/quad4/"
h5_path = work_path + "output/fault/"

In [17]:
# Function getObservations:
def getObservations(h5_path, input_set, FourierTerms):
    Observations = []
    # Loop through all Inputs
    for input_ele in input_set:
        # Open the file
        h5_file = h5_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) + "-fault.h5"
        f = h5py.File(h5_file, 'r')
        
        # Get time
        time = np.array(f['time']).reshape([-1])
        time = time - np.min(time)
        nOfTSteps = time.shape[0]

        # Get Slip rates
        SlipRates = np.array(f['vertex_fields']['slip_rate'])
        Vx = SlipRates[:, :, 0].transpose()
        Vy = SlipRates[:, :, 1].transpose()
        nOfNodes = Vx.shape[0]

        # Find the Fourier coefficients
        FourierTerms = 16
        T = np.max(time)

        # Compute the Fourier terms
        Ks = np.array(range(FourierTerms))
        coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * time)
        VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in Vx], 0)

        # Compute the fourier coefficients
        # print('time.shape: ', time.shape)
        observation = np.trapz(VxcoskPiTt, x=time)
        
        # Append the result from this file
        Observations.append(observation)
    return Observations

In [32]:
# Function runCases
def runCases(work_path, input_set):
    shell_path = work_path + "RunJobsJP.sh"
    shellRead = open(shell_path, 'r')
    list_of_lines = shellRead.readlines()
    shellRead.close()
    
    AA = [ele[0] for ele in input_set]
    BB = [ele[1] for ele in input_set]
    
    list_of_lines[9] = "AA=" + str(tuple(AA)).replace(',', '') + "\n"
    list_of_lines[10] = "BB=" + str(tuple(BB)).replace(',', '') + "\n"

    shellWrite = open(shell_path, 'w')
    shellWrite.writelines(list_of_lines)
    shellWrite.close()
    
    # Run the cases
    print('Shell path: ', shell_path)
    !source $shell_path
    
    return

In [33]:
# Example of getting observations
AA = [0.001, 0.003]
BB = [0.002, 0.004]

# Get input set
input_set = [[AA[i], BB[i]] for i in range(len(AA))]

# Run the cases
runCases(work_path, input_set)

# Get observations
res = getObservations(h5_path, input_set, 16)

Shell path:  /home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/bar_shearwave/quad4/RunJobsJP.sh
Running case A0.001_B0.002
Finished in 12 s!

Running case A0.003_B0.004
Finished in 13 s!

/home/shengduo/InverseProblems/GPRWorkingField


In [34]:
res

[array([-276.72222533,    9.21417284,    9.28709317,    9.03502741,
           9.01451615,    9.00146222,    8.60317807,    8.54807941,
           8.53598392,    7.83240749,    8.08698523,    7.65785586,
           7.0987034 ,    7.40963169,    6.62068071,    6.37692284,
        -276.2554538 ,    9.69130499,    9.73530519,    9.47562458,
           9.40846043,    9.37064061,    8.91104473,    8.81697552,
           8.72762581,    7.98022883,    8.14300881,    7.66002745,
           7.01578593,    7.25178855,    6.38254256,    6.07746384,
        -276.72222533,    9.21417284,    9.28709317,    9.03502741,
           9.01451615,    9.00146222,    8.60317807,    8.54807941,
           8.53598392,    7.83240749,    8.08698523,    7.65785586,
           7.0987034 ,    7.40963169,    6.62068071,    6.37692284]),
 array([-274.96340531,   10.8731168 ,   11.02632182,   10.64961171,
          10.70580393,   10.53617546,   10.17650899,    9.99433927,
           9.95847399,    9.07379865,    9.400